In [11]:
import os
from openai import OpenAI
from dotenv import load_dotenv

os.chdir('/Users/reejungkim/Documents/Git/working-in-progress')
load_dotenv()


client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["huggingface_read"],
)

completion = client.chat.completions.create(
    model="openai/gpt-oss-20b:novita",
    messages=[
        {
            "role": "user",
            "content": "한국은 어디에 있나요?"
        }
    ],
)

print(completion.choices[0].message)  

ChatCompletionMessage(content='한국은 동아시아에 위치해 있습니다.  \n\n- **지리적 위치**  \n  - **동쪽**: 동해(일본해)와 마주하고 있으며, 한국 반도 끝에 있는 한반도와 일본 사이에 위치합니다.  \n  - **서쪽**: 한라수도(남쪽)로 이어지는 서해(남해)와 접해 있습니다.  \n  - **남쪽**: 한국과 일본 사이에 있는 일본 바다와 마주하고 있습니다.  \n  - **북쪽**: 북한(조선민주주의인민공화국)과 한반도 경계가 이어집니다.  \n\n- **행정 구분**  \n  - **대한민국** (South Korea)은 남한 지역이며, 수도는 서울특별시입니다.  \n  - **조선민주주의인민공화국** (North Korea)은 북한 지역이며, 수도는 평양시입니다.  \n\n- **대략적인 좌표**  \n  - 위도: 약 35°~38°N  \n  - 경도: 약 126°~132°E  \n\n- **접근성**  \n  - 항공: 서울인천국제공항(인천), 김포국제공항(경기도)\n  - 해운: 부산항, 대구항, 인천항 등\n  - 철도: KTX(고속철도)를 통한 주요 도시 연결\n\n한국은 아시아 국가 중 하나로, 특히 동아시아(아동, 일본, 중국 등)과 인접해 있어 문화와 경제적 교류가 활발합니다.', role='assistant', function_call=None, tool_calls=None, reasoning_content='User asks: "한국은 어디에 있나요?" (Where is Korea?) They want a location. Provide answer in Korean likely, include details: South Korea etc. Provide some context. Also maybe mention South Korea in East Asia, etc. I\'ll answer.')


In [18]:
sample_message = completion.choices[0].message

In [19]:
import re
from typing import Dict, Any, Optional

class ChatCompletionParser:
    """ChatCompletion 응답을 깔끔하게 파싱하는 클래스"""
    
    def __init__(self):
        self.markdown_patterns = {
            'bold': r'\*\*(.*?)\*\*',
            'italic': r'\*(.*?)\*',
            'code': r'`(.*?)`',
            'list_item': r'^[-\*]\s+(.*?)$',
            'numbered_list': r'^\d+\.\s+(.*?)$',
            'header': r'^#+\s+(.*?)$'
        }
    
    def parse_message(self, message: Dict[str, Any]) -> Dict[str, Any]:
        """
        ChatCompletionMessage를 파싱하여 구조화된 정보 반환
        
        Args:
            message: ChatCompletionMessage 딕셔너리 또는 객체
            
        Returns:
            파싱된 메시지 정보
        """
        if hasattr(message, '__dict__'):
            # 객체인 경우 딕셔너리로 변환
            message_dict = message.__dict__
        else:
            message_dict = message
        
        content = message_dict.get('content', '')
        role = message_dict.get('role', 'unknown')
        
        parsed_content = self.parse_content(content)
        
        return {
            'role': role,
            'content': content,
            'parsed_content': parsed_content,
            'has_function_calls': bool(message_dict.get('function_call') or message_dict.get('tool_calls')),
            'function_call': message_dict.get('function_call'),
            'tool_calls': message_dict.get('tool_calls'),
            'reasoning_content': message_dict.get('reasoning_content')
        }
    
    def parse_content(self, content: str) -> Dict[str, Any]:
        """
        메시지 내용을 분석하여 구조화된 정보 추출
        
        Args:
            content: 메시지 내용
            
        Returns:
            파싱된 내용 정보
        """
        if not content:
            return {'text': '', 'structure': {}}
        
        lines = content.split('\n')
        structure = {
            'headers': [],
            'lists': [],
            'paragraphs': [],
            'formatting': {
                'bold_count': 0,
                'italic_count': 0,
                'code_count': 0
            }
        }
        
        current_list = None
        current_paragraph = []
        
        for line in lines:
            line = line.strip()
            
            if not line:
                if current_paragraph:
                    structure['paragraphs'].append(' '.join(current_paragraph))
                    current_paragraph = []
                continue
            
            # 헤더 감지
            if re.match(r'^#+\s+', line):
                header_match = re.match(r'^(#+)\s+(.*)', line)
                if header_match:
                    level = len(header_match.group(1))
                    text = header_match.group(2)
                    structure['headers'].append({
                        'level': level,
                        'text': text,
                        'line': line
                    })
                continue
            
            # 리스트 아이템 감지
            list_match = re.match(r'^[-\*]\s+(.*)', line)
            numbered_match = re.match(r'^\d+\.\s+(.*)', line)
            
            if list_match or numbered_match:
                item_text = list_match.group(1) if list_match else numbered_match.group(1)
                list_type = 'bullet' if list_match else 'numbered'
                
                if current_list is None or current_list['type'] != list_type:
                    if current_list:
                        structure['lists'].append(current_list)
                    current_list = {'type': list_type, 'items': []}
                
                current_list['items'].append(item_text)
                continue
            
            # 일반 텍스트 (문단)
            if current_list:
                structure['lists'].append(current_list)
                current_list = None
            
            current_paragraph.append(line)
        
        # 마지막 처리
        if current_list:
            structure['lists'].append(current_list)
        if current_paragraph:
            structure['paragraphs'].append(' '.join(current_paragraph))
        
        # 포매팅 요소 카운트
        structure['formatting']['bold_count'] = len(re.findall(self.markdown_patterns['bold'], content))
        structure['formatting']['italic_count'] = len(re.findall(self.markdown_patterns['italic'], content))
        structure['formatting']['code_count'] = len(re.findall(self.markdown_patterns['code'], content))
        
        return {
            'text': content,
            'structure': structure,
            'word_count': len(content.split()),
            'char_count': len(content)
        }
    
    def format_output(self, parsed_message: Dict[str, Any], style: str = 'clean') -> str:
        """
        파싱된 메시지를 원하는 스타일로 포맷팅
        
        Args:
            parsed_message: parse_message()의 결과
            style: 'clean', 'structured', 'minimal' 중 선택
            
        Returns:
            포맷팅된 문자열
        """
        content = parsed_message['parsed_content']
        
        if style == 'minimal':
            return self._clean_markdown(content['text'])
        
        elif style == 'structured':
            return self._format_structured(parsed_message)
        
        else:  # clean (기본값)
            return self._format_clean(content['text'])
    
    def _clean_markdown(self, text: str) -> str:
        """마크다운 문법 제거하여 깔끔한 텍스트로 변환"""
        # 볼드 제거
        text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
        # 기울임 제거
        text = re.sub(r'\*(.*?)\*', r'\1', text)
        # 코드 제거
        text = re.sub(r'`(.*?)`', r'\1', text)
        # 리스트 마커 정리
        text = re.sub(r'^[-\*]\s+', '• ', text, flags=re.MULTILINE)
        text = re.sub(r'^\d+\.\s+', '', text, flags=re.MULTILINE)
        # 헤더 마커 제거
        text = re.sub(r'^#+\s+', '', text, flags=re.MULTILINE)
        
        return text.strip()
    
    def _format_clean(self, text: str) -> str:
        """읽기 좋게 포맷팅"""
        lines = text.split('\n')
        formatted_lines = []
        
        for line in lines:
            line = line.strip()
            if not line:
                formatted_lines.append('')
                continue
            
            # 헤더는 그대로 유지하되 스타일링
            if re.match(r'^#+', line):
                formatted_lines.append(f"\n{line}\n" + "=" * len(line))
            # 리스트 아이템 정리
            elif re.match(r'^[-\*]\s+', line):
                formatted_lines.append(f"  {line}")
            else:
                formatted_lines.append(line)
        
        return '\n'.join(formatted_lines)
    
    def _format_structured(self, parsed_message: Dict[str, Any]) -> str:
        """구조화된 정보를 보기 좋게 출력"""
        content = parsed_message['parsed_content']
        structure = content['structure']
        
        output = []
        output.append(f"📝 메시지 정보")
        output.append(f"역할: {parsed_message['role']}")
        output.append(f"글자 수: {content['char_count']}")
        output.append(f"단어 수: {content['word_count']}")
        output.append("")
        
        if structure['headers']:
            output.append("📋 헤더:")
            for header in structure['headers']:
                indent = "  " * (header['level'] - 1)
                output.append(f"{indent}• {header['text']}")
            output.append("")
        
        if structure['lists']:
            output.append("📌 리스트:")
            for i, lst in enumerate(structure['lists'], 1):
                output.append(f"  리스트 {i} ({lst['type']}):")
                for item in lst['items']:
                    output.append(f"    • {item}")
            output.append("")
        
        if structure['paragraphs']:
            output.append("📄 문단:")
            for i, paragraph in enumerate(structure['paragraphs'], 1):
                preview = paragraph[:100] + "..." if len(paragraph) > 100 else paragraph
                output.append(f"  {i}. {preview}")
            output.append("")
        
        formatting = structure['formatting']
        if any(formatting.values()):
            output.append("✨ 포맷팅:")
            if formatting['bold_count']:
                output.append(f"  • 굵은 글씨: {formatting['bold_count']}개")
            if formatting['italic_count']:
                output.append(f"  • 기울임: {formatting['italic_count']}개")
            if formatting['code_count']:
                output.append(f"  • 코드: {formatting['code_count']}개")
        
        return '\n'.join(output)

# 사용 예시
def demo_usage():
    """사용 예시를 보여주는 함수"""
    
    
    # 파서 초기화
    parser = ChatCompletionParser()
    
    # 메시지 파싱
    parsed = parser.parse_message(sample_message)
    
    # 다양한 스타일로 출력
    print("=" * 60)
    print("1. 깔끔한 스타일 (clean)")
    print("=" * 60)
    print(parser.format_output(parsed, 'clean'))
    
    print("\n" + "=" * 60)
    print("2. 최소 스타일 (minimal)")
    print("=" * 60)
    print(parser.format_output(parsed, 'minimal'))
    
    print("\n" + "=" * 60)
    print("3. 구조화된 스타일 (structured)")
    print("=" * 60)
    print(parser.format_output(parsed, 'structured'))

if __name__ == "__main__":
    demo_usage()

1. 깔끔한 스타일 (clean)
한국은 동아시아에 위치해 있습니다.

  - **지리적 위치**
  - **동쪽**: 동해(일본해)와 마주하고 있으며, 한국 반도 끝에 있는 한반도와 일본 사이에 위치합니다.
  - **서쪽**: 한라수도(남쪽)로 이어지는 서해(남해)와 접해 있습니다.
  - **남쪽**: 한국과 일본 사이에 있는 일본 바다와 마주하고 있습니다.
  - **북쪽**: 북한(조선민주주의인민공화국)과 한반도 경계가 이어집니다.

  - **행정 구분**
  - **대한민국** (South Korea)은 남한 지역이며, 수도는 서울특별시입니다.
  - **조선민주주의인민공화국** (North Korea)은 북한 지역이며, 수도는 평양시입니다.

  - **대략적인 좌표**
  - 위도: 약 35°~38°N
  - 경도: 약 126°~132°E

  - **접근성**
  - 항공: 서울인천국제공항(인천), 김포국제공항(경기도)
  - 해운: 부산항, 대구항, 인천항 등
  - 철도: KTX(고속철도)를 통한 주요 도시 연결

한국은 아시아 국가 중 하나로, 특히 동아시아(아동, 일본, 중국 등)과 인접해 있어 문화와 경제적 교류가 활발합니다.

2. 최소 스타일 (minimal)
한국은 동아시아에 위치해 있습니다.  

• 지리적 위치  
  - 동쪽: 동해(일본해)와 마주하고 있으며, 한국 반도 끝에 있는 한반도와 일본 사이에 위치합니다.  
  - 서쪽: 한라수도(남쪽)로 이어지는 서해(남해)와 접해 있습니다.  
  - 남쪽: 한국과 일본 사이에 있는 일본 바다와 마주하고 있습니다.  
  - 북쪽: 북한(조선민주주의인민공화국)과 한반도 경계가 이어집니다.  

• 행정 구분  
  - 대한민국 (South Korea)은 남한 지역이며, 수도는 서울특별시입니다.  
  - 조선민주주의인민공화국 (North Korea)은 북한 지역이며, 수도는 평양시입니다.  

• 대략적인 좌표  
  - 위도: 약 35°~38°N  
  - 경도: 약 126

In [21]:
# 파서 초기화
parser = ChatCompletionParser()

# 메시지 파싱
parsed = parser.parse_message(sample_message)

# 깔끔하게 출력
clean_text = parser.format_output(parsed, 'clean')
print(clean_text)

한국은 동아시아에 위치해 있습니다.

  - **지리적 위치**
  - **동쪽**: 동해(일본해)와 마주하고 있으며, 한국 반도 끝에 있는 한반도와 일본 사이에 위치합니다.
  - **서쪽**: 한라수도(남쪽)로 이어지는 서해(남해)와 접해 있습니다.
  - **남쪽**: 한국과 일본 사이에 있는 일본 바다와 마주하고 있습니다.
  - **북쪽**: 북한(조선민주주의인민공화국)과 한반도 경계가 이어집니다.

  - **행정 구분**
  - **대한민국** (South Korea)은 남한 지역이며, 수도는 서울특별시입니다.
  - **조선민주주의인민공화국** (North Korea)은 북한 지역이며, 수도는 평양시입니다.

  - **대략적인 좌표**
  - 위도: 약 35°~38°N
  - 경도: 약 126°~132°E

  - **접근성**
  - 항공: 서울인천국제공항(인천), 김포국제공항(경기도)
  - 해운: 부산항, 대구항, 인천항 등
  - 철도: KTX(고속철도)를 통한 주요 도시 연결

한국은 아시아 국가 중 하나로, 특히 동아시아(아동, 일본, 중국 등)과 인접해 있어 문화와 경제적 교류가 활발합니다.
